In [3]:
import pandas as pd

UDPLag = pd.read_csv('./03-11/UDPLag.csv')
UDP = pd.read_csv('./03-11/UDP.csv')
LDAP = pd.read_csv('./03-11/LDAP.csv')
MSSQL = pd.read_csv('./03-11/MSSQL.csv')
NetBIOS = pd.read_csv('./03-11/NetBIOS.csv')
Portmap = pd.read_csv('./03-11/Portmap.csv')
Syn = pd.read_csv('./03-11/Syn.csv')
frames=[UDPLag,UDP,LDAP,MSSQL,NetBIOS,Portmap,Syn]

#frames=[LDAP]
trainData = pd.concat(frames)

MemoryError: Unable to allocate 1.59 GiB for an array with shape (37, 5775786) and data type int64

In [ ]:
import numpy as np
def cleanData(trainData_):
  colum_=trainData_.columns
  columns=[]

  for col in colum_:
    #col_=col.replace('/', '_')
    #col_=col_.replace(' ', '_')
    columns.append(col.replace('/', '_'))
    #print(col)

  trainData_.columns=columns;

  trainData_.dropna()
  np.nan_to_num(trainData_)
  trainData_.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)
  trainData_.replace(np.nan,0)
  row_mask=trainData_.isnull().any(axis=1)
  col_mask=trainData_.isnull().any(axis=0) 
  rowNumber=0
  dropRowIndexList=[]
  for i in row_mask:
    if i==True:
      print("droping row",rowNumber,i)
      trainData_.drop(trainData_.index[[rowNumber-1,rowNumber+1]],axis=0)
      dropRowIndexList.append(rowNumber)
      #print("Sucess")
    rowNumber=rowNumber+1   
  #trainData_.drop(dropRowIndexList)
  trainData_=trainData_[~trainData_.isin([np.nan, np.inf, -np.inf]).any(1)]

  print("Sucess")
  return trainData_

In [ ]:
print(trainData.isnull().sum().sum())
trainData=cleanData(trainData)
print(trainData.isnull().sum().sum())

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
X=trainData.drop(" Label",1)
X=X.drop("Flow ID",1)
X=X.drop(" Source IP",1)
X=X.drop(" Destination IP",1)
X=X.drop(" Timestamp",1)
X=X.drop("SimillarHTTP",1)
#X=X.dropna()

Y=trainData[" Label"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=3000).fit(X_train, Y_train)

In [ ]:
clf.score(X_test, Y_test)